# TODO
    1. Welke dimensies moeten worden opgevraagd en welke berekend?
    2. Gegeven de opgevraagde dimensies, bereken de overige relevante.
    3. Voor een comparable featuere i.e. toilet: doe een voorspelling met en een voorspelling zonder het toilet.
    4. Kijk naar de resulterende voorspellingen, het verschil is de verwachte toegevoegde waarde
    5. Gebruik fci om de confidence interval te geven!

In [1]:
!pip install forestci

In [2]:
import numpy as np
import pandas as pd
import pickle
import numbers
import statsmodels.api as sm
import random
from sklearn import datasets
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from tkinter import *
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from preprocessing import *
from sklearn.ensemble import RandomForestRegressor as RFR
from preprocessing import *
import forestci as fci

pd.options.display.max_seq_items = 2000

/home/stefan/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Failed to import duecredit due to No module named 'duecredit'


For earlier analysis, we took the log of the endprice, this time however, we are interested in the effect of certain variables on  the real endprice.

In [3]:
df = pickle.load(open('houses.p','rb'))
train_cats(df)
# df['endprice'] = np.exp(df['endprice'])

We start with a few comparable features, we should add some more in the future

In [4]:
compare_features = ['bathroom.badkamer','feature.zwembad','bathroom.aparte toilet','balcony.balkon','feature.sauna']

In [5]:
df[compare_features].isnull().sum()

bathroom.badkamer         0
feature.zwembad           0
bathroom.aparte toilet    1
balcony.balkon            0
feature.sauna             0
dtype: int64

In [6]:
df = df.dropna(subset=compare_features)

In [7]:
weights = {"livingspace" : 0.377048,
           "Gemiddelde woningwaarde:x 1 000 euro" : 0.148191,
           "housetype" : 0.116821,
           "V1.x":0.031711,
           "lotsurface":0.045889,
           "yearofconstruction":0.024880,
           "longitude":0.016243,
           'latitude':0.013181,
           'housesubtype':0.018080,
           'rooms' : 0.016288
        }
rcf = list(weights.keys())

In [15]:
print(rcf)

['livingspace', 'Gemiddelde woningwaarde:x 1 000 euro', 'housetype', 'V1.x', 'lotsurface', 'yearofconstruction', 'longitude', 'latitude', 'housesubtype', 'rooms']


In [8]:
df[rcf].isnull().sum()

livingspace                              4575
Gemiddelde woningwaarde:x 1 000 euro     1866
housetype                                   0
V1.x                                        0
lotsurface                              24943
yearofconstruction                       4288
longitude                                   0
latitude                                    0
housesubtype                                0
rooms                                       0
dtype: int64

In [9]:
weights = {"livingspace" : 0.377048,
           "Gemiddelde woningwaarde:x 1 000 euro" : 0.148191,
           "housetype" : 0.116821,
           "V1.x":0.031711,
           "lotsurface":0.045889,
           "yearofconstruction":0.024880,
           "longitude":0.016243,
           'latitude':0.013181,
           'housesubtype':0.018080,
           'rooms' : 0.016288
        }

#Normalize weights 
sum_vals = sum(weights.values())
for key in weights.keys():
    weights[key] = weights[key] / sum_vals
    
rcf = list(weights.keys())
#ccf = ["housesubtype","housetype"]
#ccf = ['feature.zwembad','feature.sauna', 'feature.glasvezelkabel']
#       'feature.stoomcabine', 'feature.stoomcabine','feature.jacuzzi']

In [10]:
df[df['lotsurface'].isnull()].housetype.value_counts()

appartement        19794
eengezinswoning     4409
herenhuis            310
villa                168
bungalow             146
woonboerderij         65
landhuis              28
woonboot              14
grachtenpand           8
stacaravan             1
landgoed               0
Name: housetype, dtype: int64

The column with the most missing values is "lotsurface". Looking at how these missing values are centered, we see that most of these missing values happen for appartments (which is to be expected). As it turns out, all appartments have lotsurface Nan. For the purpose of the regression we put this value to 0 for all appartments.

For all the other values, in order to empower regression, we could either drop the row or set the value to the average of the housetype. Since there appears to be plenty of data without Nans and since we want to use mostly accurate data, we decide to drop them.

In [11]:
df['lotsurface'] = np.where(df['housetype'] == 'appartement', 0, df['lotsurface'])
df = df.dropna(subset=rcf)

In [14]:
pickle.dump(df[rcf + compare_features + ['endprice']] ,open('../Data/reduced_df.p','wb'))

We are still left with 66239 rows, which should be plenty of data for our model.

In [ ]:
# weights --> dictionary of feature name to importance
# data --> pandas datafram of houses with all relevant features
# rcf --> list of all relevant features 
# puf --> list of all potentially upgradable features -should be a subset of relevant features 
# thres --> maximum distance a house can have from the target house if it is to be put in a reference set 
# target_house

def normalize_column(x):
    min_max_scaler = MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    return x_scaled, min_max_scaler

def weightedL2(a,b,w):
    dif = []
    for i, val in enumerate(a[0]):
        if(type(val)==type("a")):
            if val == b[i]:
                dif.append(0)
            else:
                dif.append(1)
        else:
            dif.append(abs(a[0][i]-b[i]))
    q = dif
    return np.sqrt((w*q*q).sum())

# rcf --> relevent comparable features rcfv--> relevant comparable feature values puf --> potential upgradable features
def get_reference_indices(weights, data, thres, target_values):
    reference_indices = []
    for i, house in enumerate(data):
        if weightedL2(target_values, house,weights) <  thres:
            reference_indices.append(i)
    return reference_indices

def get_reference_sets(weights, rcf, ccf, data, target_house, thres=1):
    df_ans = data.copy()
    df = data[rcf+ccf] # We only look only at the columns that were determined to be relevant for comparison
    t_house = target_house[rcf+ccf]
#     for col in ccf:
#         val = t_house[col].iloc[0]
#         df = df[df[col]==val]
    for col in df[rcf].columns:
        if (df[col].dtype in [bool,float,int,np.int64]):
            a, b = normalize_column(df[[col]].values.astype(float))
            df[col] = a
            t_house[col] = b.transform(t_house[[col]].values.astype(float))
    target_values = t_house[rcf].values
    data_array = df[rcf].values
#     return target_values
    weights_arr = np.array(list(weights.copy().values()))
    reference_index = get_reference_indices(weights_arr, data_array, thres, target_values)
    return df_ans.iloc[reference_index]

In [ ]:
# thresh is a measure of how close a house has to be to the target house in order for it to be similar
thresh = 1
t_house = pd.DataFrame(df.iloc[2]).T

In [ ]:
ref_sets = get_reference_sets(weights=weights, rcf = rcf, ccf = compare_features, data= df, target_house= t_house,thres=thresh)

In [ ]:
def feature_importance (reference_sets, rcf, compare_features):
    #This function will fit a simple regression in the data and return the feature importances for each upgradable feature
    # It will also return the test R2 score
    #drop NaN values #note we took the easy approach and dropped everything 
    df_result = reference_sets[rcf + compare_features + ['endprice']].dropna()
    #scaling the data
#     scaler = StandardScaler()
#     scaler.fit (df_result[rcf + compare_features])
    #creating X,y and spliting the dataset into train and test set
#     X = scaler.transform(df_result[rcf + compare_features])
    X = df_result[rcf + compare_features]
    y = df_result.endprice
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    #fitting the regression
    reg = LinearRegression().fit(X_train, y_train)
    score = reg.score(X_test, y_test)
    feature_imp = pd.Series(data=reg.coef_, index = rcf+compare_features)
    return (score,feature_imp)

### So we're gonna try to do this based on the randomforrestregression

Convert string data to categorical data

In [ ]:
data = ref_sets[rcf + compare_features + ['endprice']].dropna()
train_cats(data)
X, y, _ = proc_df(data, 'endprice')

n_trn = len(X) // 2
n_valid = n_trn + (len(X) // 4)
X_train, X_valid, X_test = split_vals_test(X, n_trn, n_valid)
y_train, y_valid, y_test = split_vals_test(y, n_trn, n_valid)

In [ ]:
train_cats(df)
X_2, y_2, _ = proc_df(df, 'endprice')

n_trn_2 = len(X_2) // 2
n_valid_2 = n_trn_2 + (len(X_2) // 4)
X_train_2, X_valid_2, X_test_2 = split_vals_test(X_2, n_trn, n_valid)
y_train_2, y_valid_2, y_test_2 = split_vals_test(y_2, n_trn, n_valid)

#### Let's compare the results of our subset with the result of the entire set.

In [ ]:
m = RFR(n_estimators=80, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m, X_train, y_train, X_valid, y_valid)

In [ ]:
m.fit(X_train_2, y_train_2)
error_margins = fci.random_forest_error(m, X_train_2, X_test_2)
print_score(m, X_train_2, y_train_2, X_valid_2, y_valid_2)

In [ ]:
len(error_margins)
len(X_test_2)

In [ ]:
# This function returns the r-squared, as well as the lower bound of the 95% confidence interval 
# and the upper bound of the 95% confidence interval.
def evaluate_ols(reference_set, rel_features, comp_features):
        data = reference_set[rel_features + comp_features + ['endprice']]
        data = data.astype(float)
        X = np.asarray(data[rel_features + comp_features])
        y = np.asarray(data['endprice'])
        X2 = sm.add_constant(X)
        est = sm.OLS(y, X2)
        est2 = est.fit()
        r = est2.rsquared
        lb = np.array(est2.conf_int())[:,0]
        ub = np.array(est2.conf_int())[:,1]
        return r, lb, ub
    
def predict_forest(data, target_house, model):
    train_cats(data)
    train_cats(target_house)
    X, y, _ = proc_df(data, 'endprice')
    target_house, _, _ = proc_df(target_house)
    model.fit(X, y)
    ans = model.predict(target_house)
    return ans
    
def evaluate_model(weights, rel_feats, comp_feats, data, target_houses, thres=1):
    m = RFR(n_estimators=80, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
    df = data.copy()
    predictions = []
    real_vals = []
#     For each house in target_houses we calculate the difference between our predicted endprice and the actual endprice.
    for i, row in target_houses.iterrows():
        real_val = target_houses[target_houses.index.isin([i])]['endprice'].iloc[0]
        t_house = target_houses[target_houses.index.isin([i])][rel_feats + comp_feats]
        ref_set = get_reference_sets(weights, rel_feats, comp_feats, df, t_house, thres=1)
        ref_set = ref_set[rel_feats + comp_feats + ['endprice']].dropna()
#         return ref_set
#       Calculate the predicted end price for the subset with reduced dimensions.
        pred_endprice = predict_forest(ref_set, t_house, m)
        predictions.append(pred_endprice[0])
        real_vals.append(real_val)
    return predictions, real_vals
    
def euclidean_distance_evaluation(model_pred, full_pred, actual_val):
    a = np.array(model_pred)
    b = np.array(full_pred)
    c = np.array(actual_val)
    d_1 = a-c
    ans_1 = np.sqrt(d_1.dot(d_1))
    d_2 = b-c
    ans_2 = np.sqrt(d_2.dot(d_2))
    return ans_1, ans_2

def run_evaluation(t_houses_size, weights, rel_feats, comp_feats, data, fitted_model, thres=1):
    t_houses = data.loc[random.sample(list(df.index), t_houses_size)]
    ans = evaluate_model(weights, rcf, compare_features, df, t_houses)
    m = RFR(n_estimators=80, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
    train_cats(data)
    cols = list(set(data.columns) - set(['endprice']))
    X_2, y_2, _ = proc_df(data, 'endprice')
    X_2 = X_2[cols]
    m.fit(X_2, y_2)
    train_cats(t_houses)
    t_houses, actual_values, _ = proc_df(t_houses, 'endprice')
    t_houses = t_houses[cols].fillna(0)
    fm_ans = m.predict(t_houses)
    distances = euclidean_distance_evaluation(ans[0], fm_ans, ans[1])
    return distances

By running the model a (relatively large) number of times we can see how well our predictions behave.

In [ ]:
avg_dists = run_evaluation(3, weights, rcf, compare_features, df, m)

In [ ]:
avg_dists

In [ ]:
X, y, _ = proc_df(t, 'endprice')
m.fit(X, y)
m.predict([np.array(X.iloc[0])])

We can use the standard regression we always use from scikit learn but it's hard to gain insights from that.

In [ ]:
score, importance_df = feature_importance (ref_sets, rcf, compare_features)
print ("Regression R2 score : ", score,'\n\nfeature importances : \n',importance_df)

We can also use another package, which has slightly lower R-squared but does give insight into the impact of all the variables. This is similar to what we see for STATA during the lecture.

In [ ]:
m = RFR(n_estimators=80, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m, X_train, y_train, X_valid, y_valid)

In [ ]:
data = df[rcf + compare_features + ['endprice']].dropna()
data = data.astype(float)
X = np.asarray(data[rcf + compare_features])
y = np.asarray(data['endprice'])
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())